In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Projects\\Projects\\Machine learning\\Red_wine_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path :Path
    metric_file_name :Path
    all_params:dict 
    target_column: str
    
    
        
        

In [5]:
from Red_Wine_Prediction.constants import * 
from Red_Wine_Prediction.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        
        create_directories([config.root_dir])
        model_evaluation_config= ModelEvaluationConfig(
        
        
        root_dir = config.root_dir,
        test_data_path=config.test_data_path,
        model_path= config.model_path,
        metric_file_name = config.metric_file_name,
        all_params=params,
        target_column=schema.name 
        
        )
        
        
        return model_evaluation_config

In [7]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse 
import numpy as np 
import joblib 


In [8]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config=config 
        
    def eval_metrics(self,actual,pred):
        rmse =np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2 
    
    
    def save_results(self):
        
        test_data =pd.read_csv(self.config.test_data_path)
        model= joblib.load(self.config.model_path)
        
        
        test_x =test_data.drop([self.config.target_column],axis=1)
        test_y= test_data[[self.config.target_column]]
        
        predicted_qualities=model.predict(test_x)
        
        (rmse,mae,r2) =self.eval_metrics(test_y,predicted_qualities)
        
        #saving them locally 
        
        score ={"rmse":rmse,"mae":mae,"r2":r2}
        
        save_json(path=Path(self.config.metric_file_name),data=score)

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2026-01-03 16:52:05,271: INFO: common: YAML file loaded successfully: config\config.yaml]
[2026-01-03 16:52:05,275: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-03 16:52:05,277: INFO: common: YAML file loaded successfully: schema.yaml]
[2026-01-03 16:52:05,279: INFO: common: Directory created at: artifacts]
[2026-01-03 16:52:05,280: INFO: common: Directory created at: artifacts/model_evaluation]
[2026-01-03 16:52:05,344: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]
